In [ ]:
#VGG 19 Model and results

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import urllib
import io
import skimage.transform

import lasagne
from lasagne.layers import InputLayer, DenseLayer, DropoutLayer
from lasagne.layers.conv import Conv2DLayer as ConvLayer
from lasagne.layers.pool import Pool2DLayer as PoolLayer
from lasagne.layers import NonlinearityLayer
from lasagne.layers import DropoutLayer
from lasagne.layers import LocalResponseNormalization2DLayer as NormLayer
from lasagne.utils import floatX
from lasagne.nonlinearities import softmax

/afs/ir.stanford.edu/users/c/j/cjbillov/cs231n/project/CS231N-FinalProject/.env/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
def prep_image(url):
    ext = url.split('.')[-1]
    try:
        im = plt.imread(io.BytesIO(urllib.urlopen(url).read()), ext)
    
        # Resize so smallest dim = 256, preserving aspect ratio
        h, w, _ = im.shape
        if h < w:
            im = skimage.transform.resize(im, (256, w*256/h), preserve_range=True)
        else:
            im = skimage.transform.resize(im, (h*256/w, 256), preserve_range=True)

        # Central crop to 224x224
        h, w, _ = im.shape
        im = im[h//2-112:h//2+112, w//2-112:w//2+112]
        rawim = np.copy(im).astype('uint8')

        # Shuffle axes to c01
        im = np.swapaxes(np.swapaxes(im, 1, 2), 0, 1)

        # Convert to BGR
        im = im[::-1, :, :]
        im = im - MEAN_IMAGE[:,None,None]
        return rawim, floatX(im[np.newaxis])
    except:
        # Abort
        print "skipping url " + url
        return None, np.empty((0,))
    

In [3]:
# Model to be trained/tested against

def build_model():
    net = {}
    net['input'] = InputLayer((None, 3, 224, 224))
    net['conv1_1'] = ConvLayer(
        net['input'], 64, 3, pad=1)
    net['conv1_2'] = ConvLayer(
        net['conv1_1'], 64, 3, pad=1)
    net['pool1'] = PoolLayer(net['conv1_2'], 2)
    net['conv2_1'] = ConvLayer(
        net['pool1'], 128, 3, pad=1)
    net['conv2_2'] = ConvLayer(
        net['conv2_1'], 128, 3, pad=1)
    net['pool2'] = PoolLayer(net['conv2_2'], 2)
    net['conv3_1'] = ConvLayer(
        net['pool2'], 256, 3, pad=1)
    net['conv3_2'] = ConvLayer(
        net['conv3_1'], 256, 3, pad=1)
    net['conv3_3'] = ConvLayer(
        net['conv3_2'], 256, 3, pad=1)
    net['conv3_4'] = ConvLayer(
        net['conv3_3'], 256, 3, pad=1)
    net['pool3'] = PoolLayer(net['conv3_4'], 2)
    net['conv4_1'] = ConvLayer(
        net['pool3'], 512, 3, pad=1)
    net['conv4_2'] = ConvLayer(
        net['conv4_1'], 512, 3, pad=1)
    net['conv4_3'] = ConvLayer(
        net['conv4_2'], 512, 3, pad=1)
    net['conv4_4'] = ConvLayer(
        net['conv4_3'], 512, 3, pad=1)
    net['pool4'] = PoolLayer(net['conv4_4'], 2)
    net['conv5_1'] = ConvLayer(
        net['pool4'], 512, 3, pad=1)
    net['conv5_2'] = ConvLayer(
        net['conv5_1'], 512, 3, pad=1)
    net['conv5_3'] = ConvLayer(
        net['conv5_2'], 512, 3, pad=1)
    net['conv5_4'] = ConvLayer(
        net['conv5_3'], 512, 3, pad=1)
    net['pool5'] = PoolLayer(net['conv5_4'], 2)
    net['fc6'] = DenseLayer(net['pool5'], num_units=4096)
    net['fc6_dropout'] = DropoutLayer(net['fc6'], p=0.5)
    net['fc7'] = DenseLayer(net['fc6_dropout'], num_units=4096)
    net['fc7_dropout'] = DropoutLayer(net['fc7'], p=0.5)
    net['fc8'] = DenseLayer(
        net['fc7_dropout'], num_units=1000, nonlinearity=None)
    net['prob'] = NonlinearityLayer(net['fc8'], softmax)

    return net

In [4]:
# Load model parameters and metadata
model = pickle.load(open('../weights/vgg19.pkl'))

CLASSES = model['synset words']
MEAN_IMAGE = model['mean value']

net = build_model()
lasagne.layers.set_all_param_values(net['prob'], model['param values'])

In [10]:
# Validation Set
index = urllib.urlopen('http://www.image-net.org/challenges/LSVRC/2012/ori_urls/indexval.html').read()
image_urls = index.split('<br>')

np.random.seed(19)
np.random.shuffle(image_urls)
image_urls = image_urls

images = np.zeros ((100, 3, 224, 224), dtype=np.float32)
i = 0

for im_url in image_urls:
    print i, im_url
    if 'static.flickr' not in im_url:
        used=False
        continue
    _, result = prep_image (im_url)
    if result.any():
        images[i] = result
        i+=1
        prev_used=True
    if i >= 100: 
        break

print images.shape
            

0 http://farm2.static.flickr.com/1396/905645209_85847a4186.jpg
1 http://farm3.static.flickr.com/2468/5821472960_f3cba8ff54.jpg
2 http://www.justphotos.ca/galleries/macro/bugs/images/035421.jpg
2 http://farm4.static.flickr.com/3339/3343713902_a49b2be8ac.jpg
3 http://farm4.static.flickr.com/3554/3554602023_b21e0d154f.jpg
4 http://farm5.static.flickr.com/4037/4300141082_8ee7cb4bf9.jpg
5 http://farm3.static.flickr.com/2677/4128046562_5b84309aff.jpg
6 http://farm4.static.flickr.com/3662/3374552636_1d5fc02f74.jpg
7 http://farm3.static.flickr.com/2205/5747671564_b2915c78fd.jpg
8 http://farm5.static.flickr.com/4031/4693953965_6b629db08c.jpg
9 http://farm5.static.flickr.com/4113/5170685559_4a7ec7dabb.jpg
10 http://images.wildmadagascar.org/thumbnails/berenty/ring_210030.jpg
10 http://www.visualphotos.com/photo/2x2668125/friends_playing_croquet_cb024797.jpg
10 http://farm1.static.flickr.com/22/32597379_308b7388d4.jpg
11 http://farm5.static.flickr.com/4017/5153223246_b8826496c9.jpg
12 http://farm

Here you can save the images to a pickled file.

In [ ]:
import cPickle as pickle
!pwd
def save_images (images, dataset="imagenet_val"):
    """ 
    Saves an array of images to file.
    
    """
    filename = dataset + "_" + str(images.shape[0])
    with open('../datasets/'+filename+'.pkl', 'w+') as f:
        pickle.dump(images, f)

def load_images (filename):
    """ 
    Loads images from file and returns it.
    """
    data = None
    with open('../datasets/' + filename, 'rb') as f:
        data = pickle.load(f)
    return data

uint8


In [11]:
%matplotlib inline
from time import time


# Process test images and print top 5 labels
def run_forward(images):
    #for i in xrange(10):
    im = images[:, :, :, :]

    print i, im.shape
    prob = np.array(lasagne.layers.get_output(net['prob'], im, deterministic=True).eval())
    print prob.shape #top5 = np.argsort(prob[0])[-1]
    # print top5
    #         plt.figure()
    #         plt.imshow(rawim.astype('uint8'))
    #         plt.axis('off')
    
t0 = time()
run_forward(images)
t1 = time()

print "100 images took {0} seconds".format(t1 - t0)

100 (100, 3, 224, 224)
(100, 1000)
100 images took 109.483766079 seconds
